In [33]:
import torch
import math
import numpy as np

print(torch.backends.mps.is_available()) #m1 GPU available?
print(torch.backends.mps.is_built()) #m1 GPU available?

True
True


In [21]:
#基本運算單位為Tensor,為一個多維度的矩陣,可和numpy互轉

#建立tensor
w1=torch.tensor([[1,2],[3,4],[5,6]])
w2=torch.tensor([[2,2],[2,2],[2,2]])
x=torch.empty(2,3) #創建空(未初始化)的tensor(2列3行)
print(x.dtype) #資料型態
print(x.size())
y=torch.ones(2)
z=torch.zeros(2,3,dtype=torch.int)
a=torch.randn(2,3,5)

# + - * / 皆為element-wise
ans1=w1+w2 
ans2=w1*w2 
print(ans1,ans2,sep='\n')

#矩陣乘法
x1=torch.tensor([[1,2],[3,4],[5,6]])
x2=torch.tensor([[1,2,3],[4,5,6]])
ans=x1.mm(x2) #二為矩陣乘法
print(ans)
ans=torch.mm(x1,x2) #兩個相等
print(ans)

#三維帶batch矩陣乘法：

#torch.bmm(a,b),tensor a 的size为(b,h,w),tensor b的size为(b,w,m) 
#也就是说两个tensor的第一维是相等的，然后第一个数组的第三维和第二个数组的第二维度要求一样，
#对于剩下的则不做要求，输出维度 （b,h,m）

#選取部分資料

print(w1[0,:]) #取得第一個row的tensor
print(w1[1:,:]) #取得第二個row以後的所有tensor
print(x[:,0]) #取得第ㄧcolumn的tensor
print(x[0,0]) #取得第一row,第一column的tensor
print(x[0,0].item()) #取得第一row,第一column的item
y1=y.numpy() #tensor to numpy
z1=z.view(3,2) #reshape
z2=z.reshape(3,2) #reshape和view功能基本一樣
z3=z.reshape(3,-1) #-1代表這個維度的個數是剩下的所有元素個數


torch.float32
torch.Size([2, 3])
tensor([[3, 4],
        [5, 6],
        [7, 8]])
tensor([[ 2,  4],
        [ 6,  8],
        [10, 12]])
tensor([[ 9, 12, 15],
        [19, 26, 33],
        [29, 40, 51]])
hello
tensor([[ 9, 12, 15],
        [19, 26, 33],
        [29, 40, 51]])
tensor([1, 2])
tensor([[3, 4],
        [5, 6]])
tensor([0., 0.])
tensor(0.)
0.0


In [37]:
#讀取資料
#(1)圖片放在同一文件夾內,且另有一文件顯示標籤
#(2)不同類別的圖片放在不同的文件夾內,且文件夾就是圖片的類別


#(1)
#需自定義一class來繼承torch.utils.data.dataset,並且要override 3個method

from torch.utils.data import Dataset, DataLoader
import torch
from torchvision import transforms
import os
from PIL import Image
import numpy as np

class TorchDataset(Dataset):
    def __init__(self,filename,image_dir): 
        # --------------------------------------------
        # Initialize paths, transforms, and so on
        # --------------------------------------------
        
        #filename:数据文件TXT：格式：imge_name.jpg label1_id
        #image_dir:图片路径：image_dir+imge_name.jpg构成图片的完整路径
        
        self.image_label_list=self.read_file(filename)
        self.image_dir=image_dir
        self.len=len(self.image_label_list)
        
    def img_loader(self,image_path):
        image = Image.open(image_path)
        return image.convert('RGB')
        
    def __getitem__(self, index):
        # --------------------------------------------
        # 1. Read one data from file (using numpy.fromfile, PIL.Image.open)
        # 2. Preprocess the data (torchvision.Transform).
        # 3. Return the data pair (e.g. image and label)
        # --------------------------------------------
        image_name,label=self.image_label_list[index]
        print(self.image_label_list[index])
        image_path=os.path.join(self.image_dir,image_name)
        print(image_path)
        img = self.img_loader(image_path)
        tranform = transforms.Compose([transforms.Resize((214, 214)), transforms.ToTensor()])
        img = tranform(img)
        label=np.array(label)
        return img,label
        
    def __len__(self):
        # --------------------------------------------
        # Indicate the total size of the dataset (資料的筆數)
        # --------------------------------------------
        return len(self.image_label_list)

    def read_file(self, filename): 
        image_label_list = []
        with open(filename, 'r') as f:
            lines = f.readlines()
            for line in lines:
                # rstrip：用来去除结尾字符、空白符(包括\n、\r、\t、' '，即：换行、回车、制表符、空格)
                content = line.rstrip().split(',') #是','而不是' ',可能因為是csv檔的關係
                name = content[0]
                labels = []
                for value in content[1:]:
                    labels.append(int(value))
                image_label_list.append((name, labels))
        return image_label_list
    
train_filename='/Users/sleepy/Documents/python/data/dogcat.csv'
image_dir='/Users/sleepy/Documents/python/data/cat&dog'
batch_size=9
epoch_num=2
train_data = TorchDataset(filename=train_filename, image_dir=image_dir)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

#DataLoader:用來產生批次訓練數據
#DataLoader(dataset,batch_size=1,shuffle=False,sampler=None,num_workers=0,collate_fn,pin_memory,drop_last=False)
for epoch in range(epoch_num):
    for batch_image, batch_label in train_loader:
        image=batch_image[0,:]
        image=image.numpy()#image=np.array(image)
        #image = image.transpose(1, 2, 0)  # 通道由[c,h,w]->[h,w,c]
        #image_processing.cv_show_image("image",image)
        print("batch_image.shape:{},batch_label:{}".format(batch_image.shape,batch_label))
        # batch_x, batch_y = Variable(batch_x), Variable(batch_y)

    
"""

#(2)
#調用torchvision.datasets.ImageFolder來處理

#ImageFolder(root,transform=None,target_transform=None,loader=default_loader)
#  transform:對PIL image 進行轉換操作
#  target_transform:對label進行轉換
#  loader:default為讀取PIL image
#
#  Attributes:
#  (1)classes: List of class names
#  (2)class_to_idx: Dict with items(class_name,class_index)
#  (3)imgs: List of (img path,class_index) tuples

#DataLoader:用來產生批次訓練數據
#DataLoader(dataset,batch_size=1,shuffle=False,sampler=None,num_workers=0,collate_fn,pin_memory,drop_last=False)

from torchvision.datasets import ImageFolder
import torch
from torchvision import transforms

transform = transforms.Compose([transforms.Resize((214, 214)), transforms.ToTensor()])
trainset = ImageFolder('/Users/sleepy/Documents/python/data', transform=transform)
print(trainset)
print(trainset.class_to_idx)
#trainset[0][0]:第一維度代表第幾張圖片第二維度為0代表圖片數據
#trainset[0][1]:第二維度為1代表label(0,1,2,...)
print(trainset[300][1])

data_loader=torch.utils.data.DataLoader(trainset,batch_size=20,shuffle=True)

"""


('cat.50.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.50.jpg
('cat.145.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.145.jpg
('cat.71.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.71.jpg
('cat.53.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.53.jpg
('dog.2236.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2236.jpg
('dog.2284.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2284.jpg
('cat.135.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.135.jpg
('cat.110.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.110.jpg
('dog.2293.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2293.jpg
batch_image.shape:torch.Size([9, 3, 214, 214]),batch_label:tensor([[0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1]])
('dog.2346.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2346.jpg
('cat.101.jpg', [0])
/Users/sleepy/Documents/python

('cat.38.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.38.jpg
('cat.68.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.68.jpg
('dog.2297.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2297.jpg
('dog.2222.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2222.jpg
('dog.2266.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2266.jpg
('cat.115.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.115.jpg
('cat.42.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.42.jpg
batch_image.shape:torch.Size([9, 3, 214, 214]),batch_label:tensor([[0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0]])
('cat.99.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.99.jpg
('dog.2348.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2348.jpg
('cat.106.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.106.jpg
('dog.2388.jpg', [1])
/Users/sleepy/Documents/python/

('cat.28.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.28.jpg
batch_image.shape:torch.Size([9, 3, 214, 214]),batch_label:tensor([[1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0]])
('cat.181.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.181.jpg
('cat.152.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.152.jpg
('cat.167.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.167.jpg
('dog.2238.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2238.jpg
('dog.2382.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2382.jpg
('dog.2277.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2277.jpg
('dog.2273.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2273.jpg
('cat.21.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.21.jpg
('dog.2392.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2392.jpg
batch_image.shape:torch.Size([9, 3, 214, 214]),

('cat.56.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.56.jpg
('cat.154.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.154.jpg
('dog.2319.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2319.jpg
('dog.2396.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2396.jpg
('cat.186.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.186.jpg
('dog.2283.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2283.jpg
batch_image.shape:torch.Size([9, 3, 214, 214]),batch_label:tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1]])
('cat.93.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.93.jpg
('dog.2208.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2208.jpg
('dog.2374.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2374.jpg
('cat.92.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.92.jpg
('cat.90.jpg', [0])
/Users/sleepy/Documents/pytho

('cat.8.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.8.jpg
('dog.2385.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2385.jpg
('cat.162.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.162.jpg
batch_image.shape:torch.Size([9, 3, 214, 214]),batch_label:tensor([[1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0]])
('dog.2397.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2397.jpg
('dog.2321.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2321.jpg
('dog.2226.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2226.jpg
('dog.2239.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2239.jpg
('dog.2352.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2352.jpg
('dog.2379.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2379.jpg
('dog.2204.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2204.jpg
('cat.19.jpg', [0])
/Users/sleepy/Documen

('dog.2322.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2322.jpg
('cat.135.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.135.jpg
('dog.2401.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2401.jpg
('cat.140.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.140.jpg
batch_image.shape:torch.Size([9, 3, 214, 214]),batch_label:tensor([[0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0]])
('cat.111.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.111.jpg
('cat.23.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.23.jpg
('cat.37.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.37.jpg
('dog.2407.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2407.jpg
('dog.2248.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2248.jpg
('cat.189.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.189.jpg
('cat.150.jpg', [0])
/Users/sleepy/Documents/pyth

batch_image.shape:torch.Size([9, 3, 214, 214]),batch_label:tensor([[0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1]])
('cat.156.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.156.jpg
('cat.159.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.159.jpg
('dog.2294.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2294.jpg
('cat.10.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.10.jpg
('cat.130.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.130.jpg
('cat.117.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.117.jpg
('dog.2384.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2384.jpg
('dog.2398.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2398.jpg
('cat.26.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.26.jpg
batch_image.shape:torch.Size([9, 3, 214, 214]),batch_label:tensor([[0],
        [0],
        [1],
        [0],
        [0],
    

batch_image.shape:torch.Size([9, 3, 214, 214]),batch_label:tensor([[0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0]])
('dog.2222.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2222.jpg
('cat.167.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.167.jpg
('cat.121.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.121.jpg
('dog.2358.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2358.jpg
('dog.2232.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2232.jpg
('dog.2333.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2333.jpg
('dog.2231.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2231.jpg
('cat.29.jpg', [0])
/Users/sleepy/Documents/python/data/cat&dog/cat.29.jpg
('dog.2382.jpg', [1])
/Users/sleepy/Documents/python/data/cat&dog/dog.2382.jpg
batch_image.shape:torch.Size([9, 3, 214, 214]),batch_label:tensor([[1],
        [0],
        [0],
        [1],
        [

"\n\n#(2)\n#調用torchvision.datasets.ImageFolder來處理\n\n#ImageFolder(root,transform=None,target_transform=None,loader=default_loader)\n#  transform:對PIL image 進行轉換操作\n#  target_transform:對label進行轉換\n#  loader:default為讀取PIL image\n#\n#  Attributes:\n#  (1)classes: List of class names\n#  (2)class_to_idx: Dict with items(class_name,class_index)\n#  (3)imgs: List of (img path,class_index) tuples\n\n#DataLoader:用來產生批次訓練數據\n#DataLoader(dataset,batch_size=1,shuffle=False,sampler=None,num_workers=0,collate_fn,pin_memory,drop_last=False)\n\nfrom torchvision.datasets import ImageFolder\nimport torch\nfrom torchvision import transforms\n\ntransform = transforms.Compose([transforms.Resize((214, 214)), transforms.ToTensor()])\ntrainset = ImageFolder('/Users/sleepy/Documents/python/data', transform=transform)\nprint(trainset)\nprint(trainset.class_to_idx)\n#trainset[0][0]:第一維度代表第幾張圖片第二維度為0代表圖片數據\n#trainset[0][1]:第二維度為1代表label(0,1,2,...)\nprint(trainset[300][1])\n\ndata_loader=torch.utils.data.DataLoad